In [7]:

import pandas as pd

def find_best_row(group):
    min_all = group.loc[(group['test_mse'] == group['test_mse'].min()) & 
                        (group['test_smape'] == group['test_smape'].min()) & 
                        (group['test_mase'] == group['test_mase'].min())]

    if not min_all.empty:
        return min_all.iloc[0]
    
    for metric_combination in [('test_mse', 'test_smape'), ('test_mse', 'test_mase'), ('test_smape', 'test_mase')]:
        min_two = group.loc[(group[metric_combination[0]] == group[metric_combination[0]].min()) & 
                            (group[metric_combination[1]] == group[metric_combination[1]].min())]
        if not min_two.empty:
            return min_two.iloc[0]
        
    return group.loc[group['test_mase'] == group['test_mase'].min()].iloc[0]

base_path = f"/home/noam.koren/multiTS/NFT/models/tests/test_trend/"
df = pd.read_excel(f'{base_path}evaluate_sinusodial_trend_on_syntatic_data.xlsx')

poly_true = df[df['Is Poly'] == True]
poly_false = df[df['Is Poly'] == False]

best_rows_poly_true = poly_true.groupby(['Lookback', 'Horizon', 'Vars']).apply(find_best_row).reset_index(drop=True)

In [11]:
print(df.shape)
print(poly_true.shape)
print(poly_false.shape)
print(best_rows_poly_true.shape)

(1266, 11)
(632, 11)
(634, 11)
(53, 11)


In [12]:

import pandas as pd

def find_best_row(group):
    min_all = group.loc[(group['test_mse'] == group['test_mse'].min()) & 
                        (group['test_smape'] == group['test_smape'].min()) & 
                        (group['test_mase'] == group['test_mase'].min())]

    if not min_all.empty:
        return min_all.iloc[0]
    
    for metric_combination in [('test_mse', 'test_smape'), ('test_mse', 'test_mase'), ('test_smape', 'test_mase')]:
        min_two = group.loc[(group[metric_combination[0]] == group[metric_combination[0]].min()) & 
                            (group[metric_combination[1]] == group[metric_combination[1]].min())]
        if not min_two.empty:
            return min_two.iloc[0]
        
    return group.loc[group['test_mase'] == group['test_mase'].min()].iloc[0]

base_path = f"/home/noam.koren/multiTS/NFT/models/tests/test_trend/"
df = pd.read_excel(f'{base_path}evaluate_sinusodial_trend_on_syntatic_data.xlsx')

poly_true = df[df['Is Poly'] == True]
poly_false = df[df['Is Poly'] == False]

best_rows_poly_true = poly_true.groupby(['Lookback', 'Horizon', 'Vars']).apply(find_best_row).reset_index(drop=True)
best_rows_combined = pd.DataFrame()

for _, row in best_rows_poly_true.iterrows():
    matching_row = poly_false[
        (poly_false['Lookback'] == row['Lookback']) & 
        (poly_false['Horizon'] == row['Horizon']) & 
        (poly_false['Vars'] == row['Vars']) & 
        (poly_false['Epochs'] == row['Epochs']) & 
        (poly_false['Stacks'] == row['Stacks']) & 
        (poly_false['Dagree'] == row['Dagree']) & 
        (poly_false['Blocks'] == row['Blocks'])
    ]
    best_rows_combined = pd.concat([best_rows_combined, pd.DataFrame([row])], ignore_index=True)
    if not matching_row.empty:
        best_rows_combined = pd.concat([best_rows_combined, matching_row.head(1)], ignore_index=True)

if best_rows_combined.empty:
    print("No matching rows were found.")
else:
    best_rows_combined.to_excel(f'{base_path}best_rows_combined.xlsx', index=False)


In [14]:
import pandas as pd

# Load the combined Excel file
best_combined = pd.read_excel('/home/noam.koren/multiTS/NFT/models/tests/test_trend/best_rows_combined.xlsx')

# Initialize the new column with NaN (Not a Number)
best_combined['Poly_Performance'] = pd.NA

# Define the metrics you want to compare. You can add more metrics here based on your actual data.
metrics = ['test_mse', 'test_smape', 'test_mase']

# Sort the dataframe for correct row comparisons
best_combined.sort_values(by=['Lookback', 'Horizon', 'Vars', 'Epochs', 'Stacks', 'Dagree', 'Blocks', 'Is Poly'], inplace=True)

# Go over each two rows and compare their metrics
for i in range(0, len(best_combined) - 1, 2):  # Step by 2 since we're comparing pairs
    row_1 = best_combined.iloc[i]
    row_2 = best_combined.iloc[i + 1]

    # Assuming row_1 is Poly=True and row_2 is Poly=False because of sorting, if not, check and swap
    if not row_1['Is Poly']:
        row_1, row_2 = row_2, row_1  # Swap if the first row is not Poly=True
    
    # Compare the metrics
    if all(row_1[metric] <= row_2[metric] for metric in metrics):
        # If all metrics of row_1 (Poly=True) are better or equal
        best_combined.at[i, 'Poly_Performance'] = 1
        best_combined.at[i + 1, 'Poly_Performance'] = 0
    elif all(row_2[metric] <= row_1[metric] for metric in metrics):
        # If all metrics of row_2 (Poly=False) are better or equal
        best_combined.at[i, 'Poly_Performance'] = 0
        best_combined.at[i + 1, 'Poly_Performance'] = -1

# Save the updated DataFrame back to Excel
best_combined.to_excel('/home/noam.koren/multiTS/NFT/models/tests/test_trend/best_rows_combined_updated.xlsx', index=False)
